In [ ]:
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass(
        prompt="Enter your OpenAI API key (required if using OpenAI): "
    )
print(os.environ["OPENAI_API_KEY"])

base_url = os.environ.get("BASE_URL")
model_name = os.environ.get("MODEL_NAME")

print(base_url, model_name)
print("langsmith api key", os.environ["LANGSMITH_API_KEY"])

In [2]:
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

model = init_chat_model(model_name, model_provider="openai", base_url=base_url)

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Chinese"),
    HumanMessage("hi!"),
]

model.invoke(messages)

AIMessage(content='嗨！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 12, 'total_tokens': 14, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-v3', 'system_fingerprint': None, 'id': 'chatcmpl-2a1055e4-e6f8-9408-9d89-3cc2f363f703', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--ec0dec59-9ce2-49c9-b430-eb5f5cac62e1-0', usage_metadata={'input_tokens': 12, 'output_tokens': 2, 'total_tokens': 14, 'input_token_details': {}, 'output_token_details': {}})

In [9]:
# 流式输出
for token in model.stream(messages):
    print(token.content, end="|")

|嗨|！||

In [6]:
# Prompt Templates 模板使用
from langchain.prompts import ChatPromptTemplate


system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)


prompt = prompt_template.invoke({"language": "Chinese", "text": "hi! Nice to meet you!"})


In [7]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Chinese', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi! Nice to meet you!', additional_kwargs={}, response_metadata={})]

In [8]:
for token in model.stream(prompt):
    print(token.content, end="|")


|嗨|！|很高兴|见到|你|！||